---
# --- Day 19: Aplenty ---
---

In [1]:
from typing_extensions import TypedDict
from typing import Optional, List, Dict, Tuple
import re
import numpy as np

## Load data

In [2]:
full_puzzle_data = True

In [3]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day19_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [4]:
class MachinePart(TypedDict):
    x: int
    m: int
    a: int
    s: int

In [5]:
class Rule:
    def __init__(self, rule_def: str):
        condition, self.outcome = rule_def.split(":")
        self.part_id = condition[0]
        self.multiplier = 1 if condition[1] == ">" else -1
        self.quantity = int(condition[2:])
        
    def execute_rule(self, part: MachinePart) -> Optional[str]:
        if (part[self.part_id] - self.quantity) * self.multiplier > 0:
            return self.outcome
        else:
            return None        

In [6]:
class Workflow:
    def __init__(self, rules_string: str):
        rules_data = rules_string.split(",")
        self.rules = [Rule(r) for r in rules_data[:-1]]
        self.final_outcome = rules_data[-1]        
    
    def execute_workflow(self, part: MachinePart) -> str:
        for r in self.rules:
            rule_outcome = r.execute_rule(part)
            if rule_outcome:
                return rule_outcome
        return self.final_outcome

In [7]:
workflows = dict()
for i, row in enumerate(data):
    if not row:
        i += 1
        break
    wid, rules_string = row.split("{")
    workflows[wid] = Workflow(rules_string[:-1])

In [8]:
parts = []
for row in data[i:]:
    v = list(map(int, re.findall(r"\d+", row)))
    parts.append(MachinePart(x=v[0],m=v[1],a=v[2],s=v[3]))

## --- Part One ---

In [9]:
sum_accepted = 0
for p in parts:
    out = workflows["in"].execute_workflow(p)
    while not out in ["A", "R"]:
        out = workflows[out].execute_workflow(p)
    if out == "A":
        sum_accepted += sum(p.values())

In [10]:
print(sum_accepted)

420739


## --- Part Two ---

#### Create Workflow dictionary differently

In [11]:
def reverse_inequality(s: str) -> str:
    signs = [">", "<"]
    i  = int("<" in s)
    part, qnt = s.split(signs[i])
    qnt = int(qnt) + 1 if i == 0 else int(qnt) - 1
    return part + signs[1-i] + str(qnt)
    
WF = dict()
for i, row in enumerate(data):
    if not row:
        i += 1
        break
    wid, rules_string = row.split("{")
    cond_strings = rules_string[:-1].split(",")
    inequalities = [c.split(":")[0] for c in cond_strings[:-1]]
    rev_inequalities = [reverse_inequality(inq) for inq in inequalities]
    destinations = [c.split(":")[1] for c in cond_strings[:-1]] + [cond_strings[-1]]

    wf_map = dict()
    for j, d in enumerate(destinations):
        conds = rev_inequalities[:j]
        if j < len(inequalities):
            conds.append(inequalities[j])
        wf_map[" and ".join(conds)] = d
    WF[wid] = wf_map

In [12]:
def gather_conditions(workflow_id: str):
    if workflow_id in ["A", "R"]:
        return [workflow_id]
    else:
        return [[k] + gather_conditions(v) for k, v in WF[workflow_id].items()]    

In [13]:
def get_paths(d, c = []):
    for a, *b in d:
        if len(b) == 1 and not isinstance(b[0], list):
            yield (b[0], " and ".join(c+[a]))
        else:
            yield from get_paths(b, c+[a])

In [14]:
conditions = gather_conditions("in")
valid_paths = [p[1] for p in get_paths(conditions) if p[0] == "A"]

In [15]:
def find_valid_support(cond: str) -> Dict[str, Tuple[int, int]]:
    signs = [">", "<"]
    support = {p: (1, 4000) for p in "xmas"}
    ineqs = cond.split(" and ")
    for ineq in ineqs:
        isn  = int("<" in ineq)
        part, qnt = ineq.split(signs[isn])
        a, b = support[part]
        if isn == 0:
            support[part] = (max(a, int(qnt)+1), b)
        else:
            support[part] = (a, min(b, int(qnt)-1))
    return support    

In [16]:
def count_combinations(s: Dict[str, Tuple[int, int]]) -> int:
    return np.prod(np.array([b-a+1 for a,b in s.values()], dtype=np.int64))

In [17]:
sum([count_combinations(find_valid_support(p)) for p in valid_paths])

130251901420382